In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam

base_df = pd.read_csv("/home/samir/Desktop/rudraAnalytics/sub_projects/churn/data/data.csv")
pd.set_option("display.max_columns", None)

df = base_df.copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)


def one_hot_encode(df, encode_set=[], dont_encode=[]):
    encoded_df = df.copy()
    for column in encode_set:
        if column in dont_encode:
            continue
        # print(len(df[column].unique()))
        if df[column].dtype == 'object':
            if len(df[column].unique()) == 2:
                # Binary encoding (0 and 1)
                encoded_df[column] = pd.get_dummies(df[column], drop_first=True)
            else:
                # One-hot encoding and using 0 and 1 instead of True and False
                one_hot_encoded = pd.get_dummies(df[column], prefix=column, drop_first=False)
                one_hot_encoded.columns = [f"{column}{i+1}" for i in range(one_hot_encoded.shape[1])]
                encoded_df = pd.concat([encoded_df, one_hot_encoded], axis=1)
                encoded_df.drop(column, axis=1, inplace=True)
                # print(encoded_df)

    return encoded_df

features = [
    'customerID',
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'tenure',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
    'MonthlyCharges',
    'TotalCharges',
    'Churn'
]
dont_label = ['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']

encoded_df = one_hot_encode(df.drop('customerID', axis=1), features, dont_label)
encoded_df['TotalCharges'].fillna(encoded_df['TotalCharges'].mean(), inplace=True)

X = encoded_df.drop('Churn', axis=1)
y = encoded_df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Make predictions on the test set
y_pred_dl = (model.predict(X_test) > 0.5).astype(int)

# Evaluate the model
accuracy_dl = accuracy_score(y_test, y_pred_dl)
print(f"Deep Learning Test Accuracy: {accuracy_dl * 100:.2f}%")

Epoch 1/20
89/89 [==============================] - 1s 3ms/step - loss: 0.5058 - accuracy: 0.7455 - val_loss: 0.4192 - val_accuracy: 0.8098
Epoch 2/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4320 - accuracy: 0.7950 - val_loss: 0.4112 - val_accuracy: 0.8048
Epoch 3/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4230 - accuracy: 0.7991 - val_loss: 0.4087 - val_accuracy: 0.8091
Epoch 4/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4181 - accuracy: 0.8024 - val_loss: 0.4096 - val_accuracy: 0.8105
Epoch 5/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4159 - accuracy: 0.8030 - val_loss: 0.4076 - val_accuracy: 0.8126
Epoch 6/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4131 - accuracy: 0.8039 - val_loss: 0.4062 - val_accuracy: 0.8098
Epoch 7/20
89/89 [==============================] - 0s 2ms/step - loss: 0.4101 - accuracy: 0.8085 - val_loss: 0.4082 - val_accuracy: 0.8091
Epoch 8/20
89/89 [==

Deep Learning Test Accuracy: 81.48%